In [1]:
from pybtex.database import parse_file
import json
from pprint import pprint
import os

In [2]:
# changes names format to a single string (ex: "Robert Downey Jr.")

def name_filtering(data,test):
    for j in test :
        dataset = []
        if 'author' in test[j].persons:
            str = 'author'
        elif 'editor' in test[j].persons: 
            str = 'editor'
        else: 
            continue
        for i in test[j].persons[str]:
            first = ""
            middle = ""
            last = ""
            line = ""
            all = ""
            if i.first_names:
                all += i.first_names[0]
            if i.middle_names:
                all += " " + i.middle_names[0]
            if i.last_names:
                all += " " + i.last_names[0]
            if i.lineage_names:
                all += " " + i.lineage_names[0]
            dataset.append(all)
        data[j] = {'names': dataset, 'year': test[j].fields['year'], 'title': test[j].fields['title']}
    return data

In [14]:
# creates links and nodes 

def setup(data,ref_list,to_import):
    jfile = {}
    jfile["nodes"] = []
    jfile["links"] = []
    for article in data :
        to_do = False
        for k in data[article]['names']:
            if k in ref_list:
                to_do = True
            if to_do:
                for i in data[article]['names']:
                    node = {}
                    node["id"] = i
                    node["lastColaboration"] = "0"
                    if i in ref_list:
                        node["group"] = find_team(i,to_import)
                    else:
                        node["group"] = 0
                    new_node = True
                    for _node in jfile["nodes"]:
                        if _node["id"] == i:
                            new_node = False
                    if new_node:
                        jfile["nodes"].append(node)
                    for j in data[article]['names'] :
                        if (i != j) :
                            new = True
                            for k in jfile["links"]:
                                if (k["source"] == j) & (k["target"] == i) :
                                    new = False
                                    break
                                elif (k["source"] == i) & (k["target"] == j) :
                                    k["nbLinks"] += 1
                                    link["lastColaboration"] = max(link["lastColaboration"],data[article]['year'])
                                    new = False
                                    break
                            if new:
                                link = {}
                                link["source"] = i
                                link["target"] = j
                                link["nbLinks"] = 1
                                link["lastColaboration"] = data[article]['year']
                                jfile["links"].append(link)
                    
                    for j in range(len(jfile["nodes"])):
                        if jfile["nodes"][j]["id"] == i:
                            
                            jfile["nodes"][j]["lastColaboration"] = max(jfile["nodes"][j]["lastColaboration"],data[article]['year'])
                            break
                    
    return jfile


In [4]:
def clean_data(data):
    str_e1 = '{\\\'{e}}'
    str_e1_maj = '{\\\'{E}}'
    str_tiret = '{-}'
    str_o1 = '{\\^{o}}'
    str_e2 = '{\\"{e}}'
    str_u1 = '{\\"{u}}'
    str_e3 = '{\\`{e}}'
    str_o2 = '{\\`{o}}'
    str_a1 = '{\\"{a}}'
    str_ae = '{\\ae}'
    str_ss = '{\\ss}'
    str_o3 = '{\\"{o}}'
    str_i1 = '{\\"{\\i}}'
    str_i2 = '{\\\'{\\i}}'
    str_a2 = '{\\\'{a}}'
    str_i3 = '{\\^{\\i}}'
    str_n1 = '{\\~{n}}'
    str_c1 = '{\\c{c}}'
    
    
    for article in data :
        for i in range(len(data[article]['names'])):
            data[article]['names'][i] = data[article]['names'][i].replace(str_e1,'é')
            data[article]['names'][i] = data[article]['names'][i].replace(str_tiret,'-')
            data[article]['names'][i] = data[article]['names'][i].replace(str_o1,'ô')
            data[article]['names'][i] = data[article]['names'][i].replace(str_e2,'ë')
            data[article]['names'][i] = data[article]['names'][i].replace(str_u1,'ü')
            data[article]['names'][i] = data[article]['names'][i].replace(str_e1_maj,'É')
            data[article]['names'][i] = data[article]['names'][i].replace(str_e3,'è')
            data[article]['names'][i] = data[article]['names'][i].replace(str_o2,'ò')
            data[article]['names'][i] = data[article]['names'][i].replace(str_a1,'ä')
            data[article]['names'][i] = data[article]['names'][i].replace(str_ae,'æ')
            data[article]['names'][i] = data[article]['names'][i].replace(str_ss,'ß')
            data[article]['names'][i] = data[article]['names'][i].replace(str_o3,'ö')
            data[article]['names'][i] = data[article]['names'][i].replace(str_i1,'ï')
            data[article]['names'][i] = data[article]['names'][i].replace(str_i2,'í')
            data[article]['names'][i] = data[article]['names'][i].replace(str_a2,'à')
            data[article]['names'][i] = data[article]['names'][i].replace(str_i3,'î')
            data[article]['names'][i] = data[article]['names'][i].replace(str_n1,'ñ')
            data[article]['names'][i] = data[article]['names'][i].replace(str_c1,'ç')

In [5]:
#save results in json file

def save(jfile):
    with open('test.json', 'w') as f:
        json.dump(jfile, f)

    pprint(jfile)

In [6]:
to_import = [['Romain','Vuillemot'],
             ['Audrey','Serna'],
             ['Christine','Michel'],
             ['Karim','Sehaba'],
             ['Élise','Lavoué'],
             ['Benoît','Encelle'],
             ['René','Chalon'],
             ['Jean-Charles','Marty'],
             ['Françoise','Sandoz-Guermond'],
             ['Aurélien','Tabard']]
ref_list = []
for i in to_import:
    ref_list += [i[0] + ' ' + i[1]]

In [7]:
def retrieve_files():
    to_import = []
    for x in os.walk(os.getcwd() + "\\bibtex_data"):
        if x[2]:
            team_name = x[0].replace(os.getcwd() + "\\bibtex_data\\", '')
            data = {'team': team_name, 'persons': []}
            for i in x[2]:
                i = i.replace('.bib','')
                data['persons'].append(i.split('_'))
            to_import.append(data)

    ref_list = []
    for i in range(len(to_import)):
        for j in to_import[i]['persons']:
            ref_list += [j[1] + ' ' + j[0]]
    
    return to_import,ref_list

In [8]:
def find_team(name,to_import):
    usable_name = list(reversed(name.split(" ")))
    for team in to_import:
        if usable_name in team["persons"]:
            return team["team"]
    return "unknown_team"

In [15]:
filtered_data = {}

to_import,ref_list = retrieve_files()

for team_names in to_import:
    for people in team_names['persons']:
        data = parse_file('bibtex_data/' + team_names['team'] + '/'+ people[0] + '_' + people[1] + '.bib').entries
        filtered_data = name_filtering(filtered_data,data)
        clean_data(filtered_data)

jfile = setup(filtered_data,ref_list,to_import)
save(jfile)

{'links': [{'lastColaboration': '2018',
            'nbLinks': 67,
            'source': 'Bertrand David',
            'target': 'René Chalon'},
           {'lastColaboration': '2016',
            'nbLinks': 19,
            'source': 'Bingxue Zhang',
            'target': 'Chuantao Yin'},
           {'lastColaboration': '2016',
            'nbLinks': 19,
            'source': 'Bingxue Zhang',
            'target': 'Bertrand David'},
           {'lastColaboration': '2016',
            'nbLinks': 13,
            'source': 'Bingxue Zhang',
            'target': 'René Chalon'},
           {'lastColaboration': '2016',
            'nbLinks': 15,
            'source': 'Bingxue Zhang',
            'target': 'Zhang Xiong'},
           {'lastColaboration': '2016',
            'nbLinks': 27,
            'source': 'Chuantao Yin',
            'target': 'Bertrand David'},
           {'lastColaboration': '2016',
            'nbLinks': 21,
            'source': 'Chuantao Yin',
            'target': 'R

            'nbLinks': 3,
            'source': 'Pierre Prablanc',
            'target': 'Vincent Roussarie'},
           {'lastColaboration': '2017',
            'nbLinks': 8,
            'source': 'Simon Durand',
            'target': 'Juan Pablo Bello'},
           {'lastColaboration': '2017',
            'nbLinks': 8,
            'source': 'Simon Durand',
            'target': 'Bertrand David'},
           {'lastColaboration': '2017',
            'nbLinks': 8,
            'source': 'Simon Durand',
            'target': 'Gaël Richard'},
           {'lastColaboration': '2017',
            'nbLinks': 8,
            'source': 'Juan Pablo Bello',
            'target': 'Bertrand David'},
           {'lastColaboration': '2017',
            'nbLinks': 8,
            'source': 'Juan Pablo Bello',
            'target': 'Gaël Richard'},
           {'lastColaboration': '2016',
            'nbLinks': 3,
            'source': 'Bingxue Zhang',
            'target': 'Wei Niu'},
           {'lastCo

            'source': 'Amel Yessad',
            'target': 'Vanda Luengo'},
           {'lastColaboration': '2017',
            'nbLinks': 2,
            'source': 'François Bouchet',
            'target': 'Élise Lavoué'},
           {'lastColaboration': '2017',
            'nbLinks': 2,
            'source': 'François Bouchet',
            'target': 'Vanda Luengo'},
           {'lastColaboration': '2017',
            'nbLinks': 2,
            'source': 'Élise Lavoué',
            'target': 'Vanda Luengo'},
           {'lastColaboration': '2017',
            'nbLinks': 1,
            'source': 'Rubiela Carrillo',
            'target': 'Clement Renaud'},
           {'lastColaboration': '2017',
            'nbLinks': 3,
            'source': 'Rubiela Carrillo',
            'target': 'Yannick Prié'},
           {'lastColaboration': '2017',
            'nbLinks': 3,
            'source': 'Rubiela Carrillo',
            'target': 'Élise Lavoué'},
           {'lastColaboration': '2017',
    

            'target': 'Françoise Sandoz-Guermond'},
           {'lastColaboration': '2002',
            'nbLinks': 1,
            'source': 'Jean-Luc Berger',
            'target': 'Patrick Prévôt'},
           {'lastColaboration': '2002',
            'nbLinks': 1,
            'source': 'Jean-Luc Berger',
            'target': 'Françoise Sandoz-Guermond'},
           {'lastColaboration': '2002',
            'nbLinks': 1,
            'source': 'Patrick Prévôt',
            'target': 'Françoise Sandoz-Guermond'},
           {'lastColaboration': '1998',
            'nbLinks': 3,
            'source': 'Françoise Sandoz-Guermond',
            'target': 'Lucia Gradinariu'},
           {'lastColaboration': '1998',
            'nbLinks': 3,
            'source': 'Françoise Sandoz-Guermond',
            'target': 'Gérard Beuchot'},
           {'lastColaboration': '1998',
            'nbLinks': 3,
            'source': 'Lucia Gradinariu',
            'target': 'Gérard Beuchot'},
           {'las

            'target': 'Neven A. ElSayed'},
           {'lastColaboration': '2018',
            'nbLinks': 4,
            'source': 'Bruce H. Thomas',
            'target': 'Ross T. Smith'},
           {'lastColaboration': '2018',
            'nbLinks': 4,
            'source': 'Bruce H. Thomas',
            'target': 'Wesley Willett'},
           {'lastColaboration': '2018',
            'nbLinks': 4,
            'source': 'Gregory F. Welch',
            'target': 'Pierre Dragicevic'},
           {'lastColaboration': '2018',
            'nbLinks': 4,
            'source': 'Gregory F. Welch',
            'target': 'Niklas Elmqvist'},
           {'lastColaboration': '2018',
            'nbLinks': 4,
            'source': 'Gregory F. Welch',
            'target': 'Pourang Irani'},
           {'lastColaboration': '2018',
            'nbLinks': 4,
            'source': 'Gregory F. Welch',
            'target': 'Yvonne Jansen'},
           {'lastColaboration': '2018',
            'nbLinks': 4

            'target': 'Aurélien Tabard'},
           {'lastColaboration': '2013',
            'nbLinks': 6,
            'source': 'Henri Palleis',
            'target': 'Sarah Tausch'},
           {'lastColaboration': '2013',
            'nbLinks': 6,
            'source': 'Henri Palleis',
            'target': 'Emanuel Zezschwitz'},
           {'lastColaboration': '2013',
            'nbLinks': 6,
            'source': 'Henri Palleis',
            'target': 'Franziska Schwamb'},
           {'lastColaboration': '2013',
            'nbLinks': 6,
            'source': 'Henri Palleis',
            'target': 'Heinrich Hussmann'},
           {'lastColaboration': '2013',
            'nbLinks': 6,
            'source': 'Henri Palleis',
            'target': 'Andreas Butz'},
           {'lastColaboration': '2013',
            'nbLinks': 6,
            'source': 'Hendrik Richter',
            'target': 'Simon Stusak'},
           {'lastColaboration': '2013',
            'nbLinks': 6,
          

            'source': 'Alain Mille',
            'target': 'Benoît Mathern'},
           {'lastColaboration': '2014',
            'nbLinks': 5,
            'source': 'Olivier L. Georgeon',
            'target': 'Benoît Mathern'},
           {'lastColaboration': '2014',
            'nbLinks': 8,
            'source': 'Blandine Ginon',
            'target': 'Le Vinh Thai'},
           {'lastColaboration': '2014',
            'nbLinks': 8,
            'source': 'Le Vinh Thai',
            'target': 'Stéphanie Jean-Daubias'},
           {'lastColaboration': '2014',
            'nbLinks': 8,
            'source': 'Le Vinh Thai',
            'target': 'Marie Lefèvre'},
           {'lastColaboration': '2014',
            'nbLinks': 3,
            'source': 'Le Vinh Thai',
            'target': 'Pierre-Antoine Champin'},
           {'lastColaboration': '2014',
            'nbLinks': 4,
            'source': 'Lionel Médini',
            'target': 'Pierre-Antoine Champin'},
           {'lastCola

            'target': 'Amedeo Napoli'},
           {'lastColaboration': '2008',
            'nbLinks': 11,
            'source': 'Sylvie Desprès',
            'target': 'Yannick Toussaint'},
           {'lastColaboration': '2008',
            'nbLinks': 11,
            'source': 'Stéphanie Jean-Daubias',
            'target': 'Jean Lieber'},
           {'lastColaboration': '2008',
            'nbLinks': 11,
            'source': 'Stéphanie Jean-Daubias',
            'target': 'Thomas Meilender'},
           {'lastColaboration': '2008',
            'nbLinks': 11,
            'source': 'Stéphanie Jean-Daubias',
            'target': 'Alain Mille'},
           {'lastColaboration': '2008',
            'nbLinks': 11,
            'source': 'Stéphanie Jean-Daubias',
            'target': 'Emmanuel Nauer'},
           {'lastColaboration': '2008',
            'nbLinks': 11,
            'source': 'Stéphanie Jean-Daubias',
            'target': 'Amedeo Napoli'},
           {'lastColaboration': '2

           {'lastColaboration': '2017',
            'nbLinks': 4,
            'source': 'Xavier Dolques',
            'target': 'Marc Benoît'},
           {'lastColaboration': '2017',
            'nbLinks': 4,
            'source': 'Laura Martin',
            'target': 'Alain Mille'},
           {'lastColaboration': '2017',
            'nbLinks': 4,
            'source': 'Laura Martin',
            'target': 'Marc Benoît'},
           {'lastColaboration': '2017',
            'nbLinks': 4,
            'source': 'Alain Mille',
            'target': 'Marc Benoît'},
           {'lastColaboration': '2017',
            'nbLinks': 4,
            'source': 'Simon L. Gay',
            'target': 'Amélie Cordier'},
           {'lastColaboration': '2015',
            'nbLinks': 3,
            'source': 'Assitan Traoré',
            'target': 'Alain Mille'},
           {'lastColaboration': '2015',
            'nbLinks': 3,
            'source': 'Assitan Traoré',
            'target': 'Hélène Tatteg

           {'lastColaboration': '2000',
            'nbLinks': 9,
            'source': 'Mathias Chaillot',
            'target': 'Diane Penel'},
           {'lastColaboration': '2000',
            'nbLinks': 9,
            'source': 'Mathias Chaillot',
            'target': 'Dominique L. Scapin'},
           {'lastColaboration': '2000',
            'nbLinks': 9,
            'source': 'Mathias Chaillot',
            'target': 'Jean-Marie Craeynest'},
           {'lastColaboration': '2000',
            'nbLinks': 9,
            'source': 'Olivier Ferret',
            'target': 'Denis Malavieille'},
           {'lastColaboration': '2000',
            'nbLinks': 9,
            'source': 'Olivier Ferret',
            'target': 'Diane Penel'},
           {'lastColaboration': '2000',
            'nbLinks': 9,
            'source': 'Olivier Ferret',
            'target': 'Dominique L. Scapin'},
           {'lastColaboration': '2000',
            'nbLinks': 9,
            'source': 'Olivier Fe